In [1]:
from virtualargo import *

import matplotlib.pyplot as plt
%matplotlib notebook
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
land_feature=cfeature.NaturalEarthFeature(category='physical',name='land',scale='50m',facecolor=[0.4,0.6,0.7])

In [2]:
#CHAMPS DE VISTESSE
filenames = {'U':"/home/datawork-lops-oh/somovar/WP1/data/GLOBAL-ANALYSIS-FORECAST-PHY-001-024/*.nc",
             'V':"/home/datawork-lops-oh/somovar/WP1/data/GLOBAL-ANALYSIS-FORECAST-PHY-001-024/*.nc"}
variables = {'U':'uo','V':'vo'}
dimensions = {'time': 'time', 'depth':'depth', 'lat': 'latitude', 'lon': 'longitude'}

In [3]:
# DEFINE THE VELOCITY FIELD OBJECT
# USAGE : obj=velocityfield(ds=filenames,var=variables,dim=dimensions,isglobal=0 or 1) 
tfield=velocityfield(ds=filenames,var=variables,dim=dimensions,isglobal=0)

In [ ]:
tfield.plot()

In [4]:
# Initiate some Argo float infos
#Number of float we want to advect
nfloats=100

#Then we must define numpy array (size nfloats) for lat,lon,depth and time
lat0=36.00
lon0=-50.00
t0=np.datetime64('2018-01-01')
lon=np.random.uniform(lon0-1.0, lon0+1.0, size=nfloats)
lat=np.random.uniform(lat0-1.0, lat0+1.0, size=nfloats)
depth=np.linspace(1.0,1.0,nfloats) #1m depth
ti=np.empty(nfloats,dtype='datetime64[D]')
ti[:]=t0

# DEFINE THE FLOAT OBJECT
# USAGE : obj=virtualfloat(lat=lat_array,lon=lon_array,depth=depth_array,
#                          time=time_array,vfield=velocity_field_object) 
myfloats=virtualfloat(lat=lat,lon=lon,depth=depth,time=ti,vfield=tfield)

In [ ]:
myfloats.plotfloat()

In [5]:
# START THE ADVECTION
# USAGE : float_object.advect(duration=days,dt_run=hours,dt_out=hours,output_file='my_advection_nXX.nc')
myfloats.advect(duration=365*2,dt_run=1./12,dt_out=24,output_file='test_20200225_a.nc')

INFO: Compiled ArgoParticleArgoVerticalMovementAdvectionRK4 ==> /tmp/parcels-500721/69a6749e6d22ef59a43b511d67382492_0.so
INFO: Temporary output files are stored in out-QQATLAVD.
INFO: You can use "parcels_convert_npydir_to_netcdf out-QQATLAVD" to convert these to a NetCDF file during the run.
100% (31536000.0 of 31536000.0) |########| Elapsed Time: 0:05:48 Time:  0:05:48


In [ ]:
# EXAMPLE FOR PLOTTING RESULTS
simu=xr.open_dataset('test_20200221_0.nc')
velo=xr.open_dataset('/home/datawork-lops-oh/somovar/WP1/data/GLOBAL-ANALYSIS-FORECAST-PHY-001-024/20191205_global-analysis-forecast-phy-001-024.nc')

pr1=ccrs.PlateCarree()
pr2=ccrs.LambertConformal(-72,35,false_easting=0.0, false_northing=0.0, secant_latitudes=None, standard_parallels=None, cutoff=-30)

fig=plt.figure()
ax3 = fig.add_subplot(1,1,1,projection=pr2)
ax3.add_feature(land_feature, edgecolor='black')
ax3.set_extent([simu.lon.min()-10, simu.lon.max()+10, simu.lat.min()-10, simu.lat.max()+10])

ax3.gridlines(linewidth=1, color='gray', alpha=0.5, linestyle=':')

velo['zos'][0][:][:].plot(cmap=plt.cm.coolwarm,
                          cbar_kwargs={'orientation': 'horizontal', 'shrink':0.5, 'pad':0.1},
                          transform=pr1)

for i in simu.traj:
    temp=simu.isel(traj=i).sortby('time')
    ax3.plot(temp['lon'][0],temp['lat'][0],'k.',transform=pr1)
    ax3.plot(temp['lon'],temp['lat'],linewidth=2,transform=pr1)


ax3.plot(lon0,lat0,'or','markersize',4,transform=pr1)
ax3.add_patch(mpatches.Rectangle(xy=[simu.lon.min()-8, simu.lat.min()-8], width=(simu.lon.max()-simu.lon.min()+16), height=(simu.lat.max()-simu.lat.min()+16),
                                    edgecolor='k',facecolor='none',linestyle=':',transform=pr1))

plt.title("")